# Merging and reshaping data

This brief section follows along with the last part of Chapter 5 of out textbook. There are two main topics. First, we want to learn about **merging** or **joining** data sets. Think of this like [xlookup](https://support.microsoft.com/en-us/office/xlookup-function-b7fd680e-6d10-43e6-84f9-88eae8bf5929) in Excel, or, more generally, SQL queries. In fact, you can do [SQL in Python](https://www.datacamp.com/community/tutorials/mysql-python), if you're familiar with it. We will also see **concatenating**, or **stacking** data. Finally, we will see how to **reshape our data**. We'll see **wide** and **long** data. 

```{margin} The Keys to Our Data
What variables (e.g. security ID and a date) uniquely identify our data? What variables are common across data sets? 
```